In [1]:
print(__doc__)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from io import StringIO

#Nombre del directorio de trabajo
direc = 'C:/Users/rvozzi/OneDrive - genommalabinternacional/Documentos/ECD/Geo/Datos_Cientificos'

#Leo el archivo de señales
signals = pd.read_csv(direc + '/eeg.dat',
                      delimiter=' ', 
                      names = ['timestamp','counter','eeg','attention','meditation','blinking'])

Automatically created module for IPython interactive environment


# 1. Preparacion de los Datos

In [2]:
signals

,timestamp,counter,eeg,attention,meditation,blinking
0,1.655925e+09,68,32,0,0,0
1,1.655925e+09,69,40,0,0,0
2,1.655925e+09,70,42,0,0,0
3,1.655925e+09,71,34,0,0,0
4,1.655925e+09,72,24,0,0,0
...,...,...,...,...,...,...
339296,1.655925e+09,12,86,41,26,0
339297,1.655925e+09,13,75,41,26,0
339298,1.655925e+09,14,91,41,26,0
339299,1.655925e+09,15,114,41,26,0


In [3]:
b = 0
vec = []
for i in range(0,len(signals)):
    vec.append(100*b + signals['counter'][i])
    if signals['counter'][i] == 99:
        b = b+1

In [4]:
signals['counter2'] = vec

In [5]:
signals

,timestamp,counter,eeg,attention,meditation,blinking,counter2
0,1.655925e+09,68,32,0,0,0,68
1,1.655925e+09,69,40,0,0,0,69
2,1.655925e+09,70,42,0,0,0,70
3,1.655925e+09,71,34,0,0,0,71
4,1.655925e+09,72,24,0,0,0,72
...,...,...,...,...,...,...,...
339296,1.655925e+09,12,86,41,26,0,339412
339297,1.655925e+09,13,75,41,26,0,339413
339298,1.655925e+09,14,91,41,26,0,339414
339299,1.655925e+09,15,114,41,26,0,339415


In [6]:
%%time

"""No todas las lecturas son capturadas,
esto se puede ver ya que counter no hace saltos de a 1 o -99 en algunos casos
por lo tanto, intento rellenar todas las que pueda"""

a = 0
for i in range(2,len(signals)-2):
    dif = signals['counter'][i] - signals['counter'][i-1]
    if dif != 1 and dif != -99:
        a = a + 1
        df1 = pd.DataFrame({
                        "timestamp": [(signals['timestamp'][i]+signals['timestamp'][i-1])/2],
                        "counter": [signals['counter'][i-1] + 1 if signals['counter'][i-1] < 99 else 0],
                        "eeg": [(signals['eeg'][i]+signals['eeg'][i-1])/2],
                        "attention": [(signals['attention'][i]+signals['attention'][i-1])/2],
                        "meditation": [(signals['meditation'][i]+signals['meditation'][i-1])/2],
                        "blinking": [(signals['blinking'][i]+signals['blinking'][i-1])/2],
                        "counter2": [signals['counter2'][i-1] + 1]
                        })
        signals = signals.append(df1)
        
        signals = signals.sort_values(by=['timestamp', 'counter2'])

Wall time: 11min 7s


# 2. Creo la Variable a Predecir

In [7]:
import datetime

#empieza a respirar profundo: 2022-06-22 20:08:18
t1 = datetime.datetime(2022, 6, 22, 20, 8, 18).timestamp() - 14391

#termina de respirar profundo: 2022-06-22 20:09:16
t2 = datetime.datetime(2022, 6, 22, 20, 9, 16).timestamp() - 14391

#empieza a respirar rapido: 2022-06-22 20:09:20
t3 = datetime.datetime(2022, 6, 22, 20, 9, 20).timestamp() - 14391

#termina de respirar rapido: 2022-06-22 20:10:14
t4 = datetime.datetime(2022, 6, 22, 20, 10, 14).timestamp() - 14391

In [8]:
#respira profundo vale 1
signals.loc[(signals['timestamp'] > t1) & (signals['timestamp'] < t2), 'resp'] = 1

#respira rapido vale 0
signals.loc[(signals['timestamp'] > t3) & (signals['timestamp'] < t4), 'resp'] = 0

#elimino todos los registros que no correspondan a respiracion profunda o rapida
signals = signals.dropna()

In [9]:
signals

,timestamp,counter,eeg,attention,meditation,blinking,counter2,resp
73031,1.655925e+09,67,-220.0,40.0,14.0,0.0,73167,1.0
73032,1.655925e+09,68,-211.0,40.0,14.0,0.0,73168,1.0
73033,1.655925e+09,69,-210.0,40.0,14.0,0.0,73169,1.0
73034,1.655925e+09,70,-220.0,40.0,14.0,0.0,73170,1.0
73035,1.655925e+09,71,-216.0,40.0,14.0,0.0,73171,1.0
...,...,...,...,...,...,...,...,...
132650,1.655925e+09,54,25.0,43.0,34.0,0.0,132754,0.0
132651,1.655925e+09,55,3.0,43.0,34.0,0.0,132755,0.0
132652,1.655925e+09,56,12.0,43.0,34.0,0.0,132756,0.0
132653,1.655925e+09,57,21.0,43.0,34.0,0.0,132757,0.0


# 3. Hago Cross Validation

In [14]:
%%time

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import confusion_matrix

clf = DecisionTreeClassifier(criterion = "entropy", max_depth = 10, min_samples_split = 7)

X = signals.drop(['timestamp', 'counter', 'counter2', 'resp'], axis=1)
y = signals['resp']

scores = cross_validate(clf, X, y, cv=100,
                        scoring='accuracy',
                        return_train_score=True)

Wall time: 18.1 s


# 4. Analizo el modelo

In [16]:
scores['test_score'].mean()

0.7199707839880611

In [17]:
scores['test_score'].std()

0.3405326395034769

In [18]:
scores['test_score'].min()

0.0

In [ ]:
"""
Se puede ver un alto valor del accuracy en test, pero una mala repetibilidad en test
"""

In [19]:
scores['train_score'].mean()

0.8381588354351163

In [20]:
scores['train_score'].std()

0.02678118466057456

In [21]:
scores['train_score'].min()

0.7762485115920712

In [ ]:
"""
mirando la diferencia entre los valores de accuracy entre train y test, se puede ver que el modelo generaliza bien.
Probaria con modelos mas complejos, buscando lograr una major repetitibilidad en las metricas de test
"""